### Problem 1
write a function that takes a file path as an argument which contains  this data. Your function must return two values (x and y) that contains the data from the file as  described

In [ ]:
import numpy as np

In [ ]:
def data(file_path):
    ''' just pass the path where the data is stored and the function will store the target variable in y variable and 
    rest of the data it will store in x variable. Right now i am not splitting data into train and test as it is not mentioned
    in the question'''
    
    # reading the file
    Y = np.loadtxt(fname =file_path,dtype = 'int',delimiter = ",",skiprows = 1,usecols = 0)
    
    X = np.loadtxt(fname =file_path, delimiter = ",",skiprows = 1 ,usecols=np.arange(1,785))
    
    
    # one hot encoding of Target variable
    Y_encoded= np.zeros([len(Y),10])
    for ind,num in enumerate(Y):
        Y_encoded[ind][num] = 1
    Y = Y_encoded
    
    
    # adjusting the shape
    x = X.reshape(X.shape[0], 28 * 28, 1)
    x = x.astype("float32") / 255
    
    
    y = Y.reshape(Y.shape[0], 10, 1)
    
    return x,y

# driver code
file_path ="/content/drive/MyDrive/mnist_train.csv"
x,y = data(file_path)

###  problem 2 , problem 3, problem 4 , problem 5 and problem 6
Below is the architecture for solving all the problems given in coursework 


In [ ]:
# base layer 
# created a base layer so that all the classes can inherit from this class

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update parameters and return input gradient
        pass

In [ ]:
# dense layer class 

class Dense(Layer):
    def __init__(self, input_size, output_size):
        # input size = no of neurons in the input
        # output_size = no. of neurons in the output 
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        # output_gradient = derivative of error w.r.t. output
        weights_gradient = np.dot(output_gradient, self.input.T) # derivative of error w.r.t. weight
        input_gradient = np.dot(self.weights.T, output_gradient) # derivative of error w.r.t input
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient # derivative of error w.r.t bias is equal to output_gradient
        return input_gradient

In [ ]:
# activation layer class

class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input)) # element wise multiplication

In [ ]:
# classes of different activations functions

class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)
        
class Softmax(Layer):
    def forward(self, input):
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_gradient, learning_rate):
        n = np.size(self.output)
        return np.dot((np.identity(n) - self.output.T) * self.output, output_gradient)


In [ ]:
#cost functions

# mean squared error

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

# binary cross entropy

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

# categorical cross entropy 

def categorical_cross_entropy(y_true, y_pred):
  return -np.sum(y_true * np.log(y_pred + 10**-100))

def categorical_cross_entropy_grad(y_true, y_pred):
  return -y_true/(y_pred + 10**-100)

# problem 2 = using no hidden layer, using softmax function and printing out the gradients 

In [ ]:
network = [
    Dense(784,10),
    Softmax()
]

def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 10, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)
                print(grad)


        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")
            
# train
train(network, mse, mse_prime,x,y, epochs=10, learning_rate=0.1)

# test
for x, y in zip(x, y):
    output = predict(network, x)
    print('pred:', np.argmax(output), '\ttrue:', np.argmax(y))

# problem 3 = including  hidden layer in our network

In [ ]:
network = [
    Dense(784,10),
    Tanh(),
    Dense(10,10),
    Softmax()
]

def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 10, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)


        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")
            
# train
train(network, categorical_cross_entropy, categorical_cross_entropy_grad,x,y, epochs=10, learning_rate=0.1)

# test
for x, y in zip(x, y):
    output = predict(network, x)
    print('pred:', np.argmax(output), '\ttrue:', np.argmax(y))

# problem 4 and problem 5 = including one more hidden layer and we can go to n hidden layer and with different different activation functions 

In [ ]:
network = [
    Dense(784,10),
    Tanh(),
    Dense(10,10),
    Sigmoid(),
    Dense(10,10),
    Softmax()
]

def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 10, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)


        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")
            
# train
train(network, mse, mse_prime,x,y, epochs=10, learning_rate=0.1)

# test
for x, y in zip(x, y):
    output = predict(network, x)
    print('pred:', np.argmax(output), '\ttrue:', np.argmax(y))

# problem 6 = adding momentum in dense layer class

In [ ]:
# base layer 
# created a base layer so that all the classes can inherit from this class

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate,momentum):
        # TODO: update parameters and return input gradient
        pass
# dense layer class 

class Dense(Layer):
    def __init__(self, input_size, output_size):
        # input size = no of neurons in the input
        # output_size = no. of neurons in the output 
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate, momentum):
        # output_gradient = derivative of error w.r.t. output
        weights_gradient = np.dot(output_gradient, self.input.T) # derivative of error w.r.t. weight
        input_gradient = np.dot(self.weights.T, output_gradient)  + momentum # derivative of error w.r.t input
        self.weights -= (learning_rate * weights_gradient)
        self.bias -= learning_rate * output_gradient  # derivative of error w.r.t bias is equal to output_gradient
        return input_gradient

# activation layer class

class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate,momentum):
        return np.multiply(output_gradient, self.activation_prime(self.input)) # element wise multiplication

# classes of different activations functions

class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)
        
class Softmax(Layer):
    def forward(self, input):
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_gradient, learning_rate,momentum):
        n = np.size(self.output)
        return np.dot((np.identity(n) - self.output.T) * self.output, output_gradient)

network = [
    Dense(784,10),
    Tanh(),
    Dense(10,10),
    Sigmoid(),
    Dense(10,10),
    Softmax()
]

def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 10, learning_rate = 0.01, momentum = 0.5 ,verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate, momentum)


        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")
            
# train
train(network, mse, mse_prime,x,y, epochs=10, learning_rate=0.1,momentum = 0.5)

# test
for x, y in zip(x, y):
    output = predict(network, x)
    print('pred:', np.argmax(output), '\ttrue:', np.argmax(y))